### Mejorando la red

In [ ]:
# Importamos las librerías.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
# En esta celda se crean las derivadas y funciones de activación para usar en la red neuronal
from sympy import *

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Esto se hace para poder visualizar las derivadas sin asignar valores reales a las variables a derivar
W1, W2, B1, B2, A1, A2, Z1, Z2, X, Y = symbols('W1 W2 B1 B2 A1 A2 Z1 Z2 X Y')

# Derivada de la función costo respecto a A2
C = (A2 - Y)**2
dC_dA2 = diff(C, A2)    # Lo que hace diff es derivar el primer símbolo respecto al segundo
print("dC_dA2 = ", dC_dA2) # 2*A2 - 2*Y

# Derivada de A2 respecto de Z2
logistic = lambda x: 1 / (1 + exp(-x))
_A2 = logistic(Z2)
dA2_dZ2 = diff(_A2, Z2)
print("dA2_dZ2 = ", dA2_dZ2) # exp(-Z2)/(1 + exp(-Z2))**2

# Derivada de Z2 respecto a A1
_Z2 = A1*W2 + B2
dZ2_dA1 = diff(_Z2, A1)
print("dZ2_dA1 = ", dZ2_dA1) # W2

# Derivada de Z2 respecto a W2
dZ2_dW2 = diff(_Z2, W2)
print("dZ2_dW2 = ", dZ2_dW2) # A1

# Derivada de Z2 respecto a B2
dZ2_dB2 = diff(_Z2, B2)
print("dZ2_dB2 = ", dZ2_dB2) # 1

# Derivada de A1 respecto de Z1
relu = lambda x: Max(x, 0)
_A1 = relu(Z1)

d_relu = lambda x: x > 0 # Pendiente es 1 para los positivos, 0 para los negativos
dA1_dZ1 = d_relu(Z1)
print("dA1_dZ1 = ", dA1_dZ1) # Z1 > 0

# Derivada de Z1 respecto a W1
_Z1 = X*W1 + B1
dZ1_dW1 = diff(_Z1, W1)
print("dZ1_dW1 = ", dZ1_dW1) # X

# Derivada de Z1 respecto a B1
dZ1_dB1 = diff(_Z1, B1)
print("dZ1_dB1 = ", dZ1_dB1) # 1

In [ ]:
# Se van a fijar valores para poder optimizar valores de L y rango. Luego, se van a graficar para ver cuáles son los más convenientes.

dataframe = pd.read_csv("students.csv")
dataframe.drop(columns = ["StudentID", "Age", "Gender", "Volunteering", "GradeClass"], inplace = True)

dataframe.loc[dataframe['GPA'] <= 2, 'GPA'] = 1
dataframe.loc[dataframe['GPA'] > 2, 'GPA'] = 0

def iniciar_pesos():
    global w_hidden, b_hidden, w_output, b_output 
    np.random.seed(0) # Importante: Inicializamos la semilla en 0 para que siempre dé los mismos valores random.
    w_hidden = np.random.rand(9, 9) * 2 - 1
    w_output = np.random.rand(1, 9) * 2 - 1

    b_hidden = np.random.rand(9, 1) * 2 - 1
    b_output = np.random.rand(1, 1) * 2 - 1


# Se extrae las columnas de entrada
all_inputs = (dataframe.iloc[:, 0:9].values)
# Se extrae la columna de salida
all_outputs = dataframe.iloc[:, -1].values

# Se divide en un conjunto de entrenamiento y uno de prueba (un tercio para el testeo)
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs, test_size=1/3)

n = X_train.shape[0] # número de registros de entrenamiento

In [ ]:
# Para graficar

import matplotlib.pyplot as plt

# Configurar los gráficos
fmt_train = {
    'color': 'tab:blue',
    'ls': 'solid',
    'lw': 3,
}

fmt_test = {
    'color': 'tab:orange',
    'ls': 'solid',
    'lw': 3,
}

# Graficar accuracy de cada combinación
def graficar_accuracy(L, train, test):
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))

    ax.plot(train, label="Train", **fmt_train)
    ax.plot(test, label="Test", **fmt_test)


    ax.grid(which="both")
    ax.legend()
    ax.set_title(f"{L=}")
    ax.set_xlabel("Iteraciones")
    ax.set_ylabel("Accuracy")

    fig.tight_layout()
    plt.show()


# Además, a la hora de ir entrenando la red tendremos que ir guardando información para realizar los gráficos.
def entrenarProbarGuardar(L, rango):
  global w_hidden, b_hidden, w_output, b_output  
  iniciar_pesos() # Para que no aprenda sobre lo aprendido

  accuracy_TEST_l = []  # Se vacían para graficar cada combinación
  accuracy_TRAIN_l = []

  for i in tqdm(range(rango)):
    # seleccionar aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2 = backward_prop(Z1, A1, Z2, A2, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_output -= L * dW2
    b_output -= L * dB2

    # Calculamos de precisión de TEST
    predictions = forward_prop(X_test.transpose())[3] # Capa de salida A2
    test_comparisons = np.equal((predictions > 0.5).flatten().round(), Y_test) # Cambiará en algo el =>
    accuracy_TEST = sum(test_comparisons.astype(int) / X_test.shape[0])
    accuracy_TEST_l.append(accuracy_TEST)

    # Calculamos presición de TRAIN
    predictions = forward_prop(X_train.transpose())[3] # Capa de salida A2
    train_comparisons = np.equal((predictions > 0.5).flatten().round(), Y_train)
    accuracy_TRAIN = sum(train_comparisons.astype(int) / X_train.shape[0])
    accuracy_TRAIN_l.append(accuracy_TRAIN)
    
  # Se grafica y compara la precisión de TEST y TRAIN
  graficar_accuracy(L,  accuracy_TRAIN_l, accuracy_TEST_l)
  
# Lista de L a probar
l_probados = [0.0005, 0.005, 0.0001, 0.001, 0.05, 0.01, 0.1]
# Lista de ranges a probar
range_probados = [1_000, 3_000, 5_000, 10_000, 20_000, 30_000, 50_000, 100_000]

# Probar cada combinación de L e iteraciones
for L in l_probados:
    for rango in range_probados:           
         entrenarProbarGuardar(L, rango)

## Uso de Scikit-Learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore", category=UserWarning) # Ojos que no ven, corazón que no siente

# Cargar y preparar datos
dataframe_ = pd.read_csv("students.csv")
dataframe_.drop(columns=["StudentID", "Age", "Gender", "Volunteering", "GradeClass"], inplace=True)
df = dataframe_
np.random.seed(0)
df['GoodStudent'] = (df.iloc[:, -1] >= 2).astype(int)
X = df.values[:, :-1]
Y = df['GoodStudent'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=42)

def graficar_red_scikit(L, rango):
    
    # Crear el modelo de red neuronal con warm_start
    red_neuronal = MLPClassifier(
        solver='sgd', hidden_layer_sizes=(9,), activation='relu',
        max_iter=1, learning_rate_init=L, warm_start=True, random_state=0)

    # Listas para almacenar las precisiones en cada iteración
    train_accuracy = []
    test_accuracy = []

    # Entrenamiento iterativo para registrar precisión en cada ciclo
    for i in range(rango):
        red_neuronal.fit(X_train, Y_train)
        train_accuracy.append(red_neuronal.score(X_train, Y_train))
        test_accuracy.append(red_neuronal.score(X_test, Y_test))

    # Graficar precisión por iteración
    plt.figure(figsize=(10, 8))
    plt.plot(train_accuracy, label="Train", **fmt_train)
    plt.plot(test_accuracy, label="Test", **fmt_test)
    plt.xlabel("Iteraciones")
    plt.ylabel("Precisión")
    plt.title(f"{L=}")
    plt.legend(loc="best")
    plt.grid()
    plt.show()

In [ ]:
# Lista de L a probar
l_probados = [0.0005, 0.005, 0.0001, 0.001, 0.05, 0.01, 0.1]
# Lista de ranges a probar
range_probados = [1_000, 3_000, 5_000, 10_000, 20_000, 30_000, 50_000, 100_000]

# Probar cada combinación de L e iteraciones
for L in l_probados:
    for rango in range_probados:   
        graficar_red_scikit(L, rango)